# REGULARIZACIÓN

### Paso1: Importar librerías 

In [1]:
import tensorflow as tf
import numpy as np
from functools import partial

2024-12-19 21:21:39.149500: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Paso2: Preparación de datos

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

### Paso3: División de datos

In [3]:

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

In [4]:
class_names = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag", "Ankle boot"]

## REGULARIZACIÓN $\ell_1$ Y $\ell_2$

### Paso 4: Creación del modelo 
Se crea un modelo con la función de regularización L2 con un factor de regularización de 0.01

In [24]:
tf.random.set_seed(42)

In [25]:
RegularizedDense = partial(tf.keras.layers.Dense,
            activation="relu",
            kernel_initializer="he_normal",
            kernel_regularizer=tf.keras.regularizers.l2(0.01))

model_l2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    RegularizedDense(100),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)
model_l2.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])

### Paso 5: Entrenamiento el modelo

In [26]:
history = model_l2.fit(X_train, y_train, epochs=2,validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7064 - loss: 4.0973 - val_accuracy: 0.8252 - val_loss: 1.8654
Epoch 2/2
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8133 - loss: 1.6283 - val_accuracy: 0.8224 - val_loss: 1.1188


### Paso 6: Evaluación el modelo

In [27]:
# evaluacion del modelo
model_l2.evaluate(X_train, y_train)

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8261 - loss: 1.1227


[1.1254523992538452, 0.8249818086624146]

In [28]:
#evaluación del mismo modelo con los datos de test
model_l2.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8190 - loss: 1.1419


[1.1475615501403809, 0.8122000098228455]

In [29]:
model_l2.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_4 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,612 (350.05 KB)

 Trainable params: 89,610 (350.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

## DROPOUT

### Paso 4: Creación del modelo 

In [30]:
tf.random.set_seed(42)

In [31]:
model_dropout = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [32]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model_dropout.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

### Paso 5: Entrenamiento el modelo

In [33]:
history = model_dropout.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6887 - loss: 0.8614 - val_accuracy: 0.8316 - val_loss: 0.4455
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8094 - loss: 0.5217 - val_accuracy: 0.8406 - val_loss: 0.4304
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8260 - loss: 0.4814 - val_accuracy: 0.8580 - val_loss: 0.3847
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8326 - loss: 0.4563 - val_accuracy: 0.8522 - val_loss: 0.3789
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8382 - loss: 0.4409 - val_accuracy: 0.8630 - val_loss: 0.3763
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8439 - loss: 0.4267 - val_accuracy: 0.8680 - val_loss: 0.3605
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8488 - loss: 0.4118 - val_accuracy: 0.8662 - val_loss: 0.3480
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8508 - loss: 0.4034 - 

### Paso 6: Evaluación el modelo con los datos de testing

In [16]:
model_dropout.evaluate(X_train, y_train)

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 946us/step - accuracy: 0.8849 - loss: 0.3073


[0.3090185821056366, 0.8840727210044861]

In [17]:
model_dropout.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.8620 - loss: 0.3628


[0.36736077070236206, 0.8618000149726868]

## MAX-NORM

### Paso 4: Creación del modelo 

In [34]:
dense = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal", kernel_constraint=tf.keras.constraints.max_norm(1.))

In [ ]:
MaxNormDense = partial(tf.keras.layers.Dense, activation="relu", kernel_initializer="he_normal", kernel_constraint=tf.keras.constraints.max_norm(1.))

tf.random.set_seed(42)
model_maxNorm = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    MaxNormDense(100),
    MaxNormDense(100),
    tf.keras.layers.Dense(10, activation="softmax")
])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model_maxNorm.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

### Paso 5: Entrenamiento el modelo

In [20]:
history = model_maxNorm.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7414 - loss: 0.7309 - val_accuracy: 0.8266 - val_loss: 0.4388
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8440 - loss: 0.4260 - val_accuracy: 0.8290 - val_loss: 0.4351
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8579 - loss: 0.3859 - val_accuracy: 0.8302 - val_loss: 0.4256
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8649 - loss: 0.3669 - val_accuracy: 0.8394 - val_loss: 0.4064
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8689 - loss: 0.3519 - val_accuracy: 0.8418 - val_loss: 0.4110
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8725 - loss: 0.3415 - val_accuracy: 0.8430 - val_loss: 0.4013
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8748 - loss: 0.3333 - val_accuracy: 0.8426 - val_loss: 0.4099
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8780 - loss: 0.3274 - 

### Paso 6: Evaluación el modelo

In [21]:
model_maxNorm.evaluate(X_train, y_train)

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8791 - loss: 0.3169


[0.31956493854522705, 0.8783454298973083]

In [22]:
model_maxNorm.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8588 - loss: 0.3920


[0.3896755278110504, 0.8565000295639038]